In [1]:
import os
os.listdir()

['.ipynb_checkpoints',
 'bird_200.json',
 'bird_dev.json',
 'bird_dev_all.json',
 'bird_dev_kn.json',
 'bird_dev_kn_2.json',
 'bird_dev_kn_3.json',
 'bird_rest.json',
 'dev_bird_2.json',
 'dev_bird_3.json',
 'dev_bird_filtered.json',
 'dev_bird_filtered_2.json',
 'dev_bird_filtered_3.json',
 'dev_spider_filtered.json',
 'dev_spider_filtered_2.json',
 'dev_spider_knowledge.json',
 'dev_spider_knowledge_2.json',
 'dev_spider_used.json',
 'dev_spider_used_2.json',
 'GENAI[1].zip',
 'main_model',
 'selecting Random 200.ipynb',
 'spider_text_sql.csv',
 'SQLNet',
 'sqlnet_inference_py36',
 'stanford-postagger-full-2020-11-17',
 'sts_bird.ipynb',
 'sts_spider.ipynb',
 'Untitled.ipynb',
 'Untitled1.ipynb',
 'Untitled2.ipynb']

In [2]:
import torch
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"

model_id = "all-MiniLM-L6-v2" 
model = SentenceTransformer(model_id).to(device=device)

In [4]:
import json

# Read existing data
with open('bird_200.json', 'r+') as f:
    using_data = json.load(f)           # read JSON
    for i in using_data:
        i["top_5"] = [] # modify it

    # f.seek(0)                     # move to file start
    # json.dump(data, f, indent=2)  # write updated JSON
    # f.truncate()                  # remove leftover old content

with open('bird_rest.json', 'r+') as f:
    knowledge_data = json.load(f)
    filtered_knowledge_data = [
    {"query": item["SQL"], "question": item["question"], "score": 0}
    for item in knowledge_data]

In [9]:
i = 1
for use in using_data:
    print(i, use["question"])
    i += 1
    for k in filtered_knowledge_data:
        sentence = [k["question"], use["question"]]
        # print(sentence)
        embeddings = model.encode(sentence)
        similarities = model.similarity(embeddings[0], embeddings[1])
        k["score"] = similarities.numpy()[0][0]
        # break
    filtered_knowledge_data.sort(key=lambda x: x["score"], reverse=True)
    use["top_5"] = [
    {"query": item["query"], "question": item["question"]}
    for item in filtered_knowledge_data[:5]]
    

1 Please list all team names which the speed class is fast.
2 How many circuits are there in Melbourne, Australia?
3 How many Formula_1 races took place on the circuits in Italy?
4 What proportion of cards do not have a text box with a normal layout?
5 What's the magic card market name for the set which was released on 2017/6/9?
6 What is the percentage of molecules that are carcinogenic?
7 What is the percentage of borderless cards?
8 Which budget allowed the most money for water, chips, and cookies?
9 For all female patient with total protein (TP) beyond the normal index, what is the deviation of their TP idex from the normal.
10 When was the set of cards with "Ancestor's Chosen" released?
11 What's Bruno Senna's Q1 result in the qualifying race No. 354?
12 What is the free rate for students between the ages of 5 and 17 at the school run by Kacey Gibson?
13 What is the district code for the School that does not offer a magnet program in the city of Fresno?
14 What's Angela Sanders's 

In [10]:
import json

with open('dev_bird_filtered_200.json', 'w') as f:
    json.dump(using_data, f, indent=2)


[{'query': "SELECT DISTINCT t1.team_long_name FROM Team AS t1 INNER JOIN Team_Attributes AS t2 ON t1.team_api_id = t2.team_api_id WHERE t2.buildUpPlaySpeedClass = 'Slow'",
  'question': 'List down the long name for slow speed class team.'},
 {'query': 'SELECT t1.buildUpPlaySpeed FROM Team_Attributes AS t1 INNER JOIN Team AS t2 ON t1.team_api_id = t2.team_api_id ORDER BY t1.buildUpPlayDribbling ASC LIMIT 4',
  'question': 'What are the speed in which attacks are put together of the top 4 teams with the highest build Up Play Speed?'},
 {'query': "SELECT DISTINCT t1.buildUpPlaySpeedClass FROM Team_Attributes AS t1 INNER JOIN Team AS t2 ON t1.team_api_id = t2.team_api_id WHERE t2.team_long_name = 'KSV Cercle Brugge'",
  'question': 'Does the KSV Cercle Brugge team have a slow, balanced or fast speed class?'},
 {'query': "SELECT T1.fastestLap FROM results AS T1 INNER JOIN races AS T2 on T1.raceId = T2.raceId WHERE T2.year = 2009 AND T1.time LIKE '_:%:__.___'",
  'question': 'What is the fas